In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
header={
    'User-Agent':'Mozilla/5.0'
}

crypto_data=[]
for page in range(1,21):
    print(f"scraping page {page}...")
    url= f'https://coinmarketcap.com/?page={page}'
    r = requests.get(url,headers=header)
    soup = BeautifulSoup(r.text, 'lxml')

    rows = soup.find_all('tr')

    records_on_page = 0
    for row in rows:
        tds = row.find_all('td')
        if len(tds) > 8:
            data = {}

            name_tag = tds[2].find('p',class_='sc-c1554bc0-0 eWrlhi coin-item-name')
            data['COIN_NAME'] = name_tag.text.strip() if name_tag else 'N/A'

            symbol_tag = tds[2].find('p',class_='sc-c1554bc0-0 gOVMPU coin-item-symbol')
            data['SYMBOL'] = symbol_tag.text.strip() if symbol_tag else 'N/A'

            price_tag = tds[3].find('span')
            data["PRICE (USD)"] = price_tag.text.strip() if price_tag else 'N/A'

            change_tag = tds[4].find('span')
            data['1h CHANGE (%)'] = change_tag.text.strip() if change_tag else 'N/A'

            day_change_tag = tds[5].find('span')
            data['24h CHANGE (%)'] = day_change_tag.text.strio() if day_change_tag else 'N/A'

            week_change_tag = tds[6].find('span')
            data['7d CHANGE (%)'] = week_change_tag.text.strip() if week_change_tag else 'N/A'

            market_cap_tag = tds[7].find('span',class_='sc-11478e5d-1 jfwGHx')
            data['MARKET CAP'] = market_cap_tag.text.strip() if market_cap_tag else 'N/A'

            volume_container = tds[8]
            volume_usd_tag = volume_container.find('a')
            if volume_usd_tag:
                volume_usd_p = volume_usd_tag.find('p')
                data['VOLUME (24H) USD'] = volume_usd_p.text.strip() if volume_usd_p else 'N/A'
            else:
                data['VOLUME (24H) USD'] = 'N/A'

            supply_tag = tds[9].find('span')
            data['CIRCULATING SUPPLY'] = supply_tag.text.strip() if supply_tag else ''

            crypto_data.append(data)
            records_on_page += 1

    print(f"records scraped on page {page}: {records_on_page}")
    time.sleep(2)

crypto_df = pd.DataFrame(crypto_data)

In [ ]:
print(crypto_df.head(10))
print(f"\n Total records scraped: {len(crypto_df)}")

In [ ]:
crypto_df.to_excel('crypto_data_analys.xlsx', index=False)